In [485]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


train = pd.read_csv(r'C:\Users\Lenovo\OneDrive\Pulpit\MyProjects\titanic\train.csv')
test = pd.read_csv(r'C:\Users\Lenovo\OneDrive\Pulpit\MyProjects\titanic\test.csv')
gender_sub = pd.read_csv(r'C:\Users\Lenovo\OneDrive\Pulpit\MyProjects\titanic\gender_submission.csv')
#train_y = train['Survived']
#test_y = test['Survived']

train_y = train['Survived']
test_y=gender_sub['Survived']



In [486]:
train = train.drop('Survived', axis=1)

In [487]:
data=[train,test]
for dataset in data:
    dataset['Sex']=dataset['Sex'].map({'male':0, 'female':1})
    dataset['Prefix'] = dataset['Name'].str.split(expand=True)[1]

In [488]:
train = train.drop('PassengerId', axis=1)
test = test.drop('PassengerId', axis=1)

In [489]:
prefix = train['Prefix'].value_counts()
prefix
top_prefixes = prefix[prefix>len(train)/25]
top_prefixes

Mr.        502
Miss.      179
Mrs.       121
Master.     40
Name: Prefix, dtype: int64

In [490]:
train['Prefix'] = train['Prefix'].apply(lambda x: x if x in top_prefixes else 'Other')
test['Prefix'] = test['Prefix'].apply(lambda x: x if x in top_prefixes else 'Other')

In [491]:
train.isnull().sum()

Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
Prefix        0
dtype: int64

In [492]:
test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
Prefix        0
dtype: int64

In [493]:
#Dropping the columns that have more than 70% of data missing
cols_null = train.columns[train.isnull().sum()>len(train)*0.7]

In [494]:
train = train.drop(cols_null, axis=1)
test = test.drop(cols_null, axis=1)


In [499]:
#we are substituting one missing value with the mean value for Fare
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())


0

In [472]:
train.Fare.describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [500]:
#creating more generalized categories for fare
train['Fare_cat'] = pd.qcut(train.Fare, q=4, labels=[1,2,3,4]) #categories based on price of ticket
test['Fare_cat'] = pd.qcut(test.Fare, q=4, labels=[1,2,3,4])
train['Fare_cat'] = train['Fare_cat'].astype('int64')
test['Fare_cat'] = test['Fare_cat'].astype('int64')

In [474]:
#now that we have fare categories, we can drop 'Fare' column
train=train.drop('Fare', axis=1)
test = test.drop('Fare', axis=1)

In [475]:
#making combined category for family on board
data=[train,test]
for dataset in data:
    dataset['Fam'] = dataset['SibSp']+dataset['Parch']
    

In [476]:
train = train.drop(['SibSp', 'Parch'], axis=1)
test = test.drop(['SibSp', 'Parch'], axis=1)

In [477]:
train.isnull().sum()

Pclass        0
Name          0
Sex           0
Age         177
Ticket        0
Embarked      2
Prefix        0
Fare_cat      0
Fam           0
dtype: int64

In [478]:
train.dtypes

Pclass        int64
Name         object
Sex           int64
Age         float64
Ticket       object
Embarked     object
Prefix       object
Fare_cat      int64
Fam           int64
dtype: object

In [479]:

obj_cats = train.columns[train.dtypes=='object']

In [480]:
obj_cats

Index(['Name', 'Ticket', 'Embarked', 'Prefix'], dtype='object')

In [481]:
num_cats = train.columns[train.dtypes!='object']

In [482]:
num_cats

Index(['Pclass', 'Sex', 'Age', 'Fare_cat', 'Fam'], dtype='object')

In [508]:
#as we got prefix we can get rid of 'Name' category
#we can also drop information about the ticket since we decided to drop 'Cabin' column due to amount of lost data
#the rest of object categories can be transformed by one hot encoding because they don't have too much unique values

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

pipeline = Pipeline([('imputer', SimpleImputer(strategy='mean')), ('one_hot', OneHotEncoder(handle_unknown='ignore'))])
